In [1]:
# notes : the model should be robust for both male and female speakers , and all age groups
# strategy 1 : fit a simple binary logistic model


# To do 

# create dataset 
# use youtube-dl to download audio, use sox to convert audio to 16k mono format 
# download robust audio samples

# create tensorflow model

# create json dataset file , and dataloader


# ________________________________________________________

# ideas 

# use MFCC, total energy and F0



In [2]:
import tensorflow as tf
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import numpy as np
import os

directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data/'
directory = os.fsencode(directory_path)

In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [15]:
# numpy function to create one hot vector

def get_one_hot(targets, nb_classes):
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]
        

In [17]:
# testing above function
get_one_hot(1,2)

array([[ 0.,  1.]])

In [4]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 1
display_step = 1

In [5]:
max_frames=2000

x = tf.placeholder(tf.float32, [None, 26000]) # mnist data image of shape max_frames*13
y = tf.placeholder(tf.float32, [None, 2]) # Happy or Sad => 2 classes

In [6]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < max_frames:
        
        diff = max_frames - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     else:
        
        return input
        

In [7]:
# Set model weights
W = tf.Variable(tf.zeros([26000, 2]))
b = tf.Variable(tf.zeros([2]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

In [8]:
# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [23]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        
         # Loop over all files
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".wav"): 
                 print(filename)
                 accoustic_features=pad(audio_to_mfcc(directory_path + filename))
                 
                 accoustic_features=np.reshape(accoustic_features, (1,26000))
                    
                 print(accoustic_features.shape)
                
                 txt_file=filename.replace('.wav','.txt')
                 with open(directory_path + txt_file, 'r') as myfile:
                    data=myfile.read().replace('\n', '')
                    y=np.float32(get_one_hot(int(data), 2))
                    print(y.dtype)
            
                    # Fit training using single example data
                    _, c = sess.run([optimizer, cost], feed_dict={x: accoustic_features,
                                                          y: y})
                    # Compute average loss
                    avg_cost += c / total_batch
            
            
        
                 continue
            else:
                continue
        
        
     
            
           
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished!")

    # Test model
 #   correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
  #  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  #  print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})

boy.wav
(1, 26000)
float32


TypeError: unhashable type: 'numpy.ndarray'

In [44]:
# create a code snippet, that iterates through whole dataset 
# dataset template, in data folder, there is abc.wav, and abc.txt , containing either 0 or 1 
# let 0 be sad, and 1 be happy


for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".wav"): 
         print(filename)
         accoustic_features=pad(audio_to_mfcc(directory_path + filename))
        # pad(accoustic_features)
         print(accoustic_features.shape)
         txt_file=filename.replace('.wav','.txt')
         with open(directory_path + txt_file, 'r') as myfile:
            data=myfile.read().replace('\n', '')
            print(tf.one_hot(int(data), 2))
    
         continue
    else:
        continue

boy.wav
(2000, 13)
Tensor("one_hot_14:0", shape=(2,), dtype=float32)
lizzie.wav
(2000, 13)
Tensor("one_hot_15:0", shape=(2,), dtype=float32)


In [40]:
# this snippet is just for testing purposes 
# ignore it

accoustic_features=audio_to_mfcc(directory_path + 'lizzie.wav')

new_features = pad(accoustic_features)

print(new_features.shape)



(1000, 13)
